<a href="https://colab.research.google.com/github/Altaieb-Mohammed/lab_2corse/blob/master/lab10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import re
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report, accuracy_score
from scipy.sparse import hstack

# 1. Загрузка датасета
url = "https://raw.githubusercontent.com/Altaieb-Mohammed/lab_2corse/master/clean_synthetic_dataset.csv"
df = pd.read_csv(url)

# 2. Очистка текста (оставляем буквы, цифры и пробелы)
def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'[^a-z0-9\s]', '', text)
    return text

df['text_clean'] = df['text'].apply(preprocess_text)

# 3. Входные данные и метки
X_text = df['text_clean']
X_num = df[['estate_worth', 'debts', 'funeral_expenses', 'bequests']]
y = df['label']

# 4. Делим на обучающую и тестовую выборки
X_text_train, X_text_test, X_num_train, X_num_test, y_train, y_test = train_test_split(
    X_text, X_num, y, test_size=0.2, random_state=42, stratify=y)

# 5. Векторизация текста TF-IDF
vectorizer = TfidfVectorizer(stop_words='english', ngram_range=(1,2), max_df=0.85, min_df=2)
X_text_train_tfidf = vectorizer.fit_transform(X_text_train)
X_text_test_tfidf = vectorizer.transform(X_text_test)

# 6. Масштабирование числовых признаков
scaler = StandardScaler()
X_num_train_scaled = scaler.fit_transform(X_num_train)
X_num_test_scaled = scaler.transform(X_num_test)

# 7. Объединяем текстовые и числовые признаки
X_train_combined = hstack([X_text_train_tfidf, X_num_train_scaled])
X_test_combined = hstack([X_text_test_tfidf, X_num_test_scaled])

# 8. Обучение модели SVM с балансировкой классов
model = LinearSVC(class_weight='balanced', random_state=42, max_iter=5000)
model.fit(X_train_combined, y_train)

# 9. Предсказания и оценка
y_pred = model.predict(X_test_combined)

print(f"Accuracy: {accuracy_score(y_test, y_pred):.4f}")
print("Classification Report:\n", classification_report(y_test, y_pred, target_names=['Low Bequest', 'High Bequest']))


Accuracy: 0.9950
Classification Report:
               precision    recall  f1-score   support

 Low Bequest       0.99      1.00      1.00       100
High Bequest       1.00      0.99      0.99       100

    accuracy                           0.99       200
   macro avg       1.00      0.99      0.99       200
weighted avg       1.00      0.99      0.99       200



In [14]:
from sklearn.svm import LinearSVC

model = LinearSVC(class_weight='balanced', random_state=42, max_iter=5000)
model.fit(X_train_combined, y_train)


LinearSVC(class_weight='balanced', max_iter=5000, random_state=42)

In [15]:
import joblib

joblib.dump(model, 'svm_model.joblib')
joblib.dump(vectorizer, 'tfidf_vectorizer.joblib')
joblib.dump(scaler, 'scaler.joblib')


['scaler.joblib']

In [16]:
import joblib
from sklearn.svm import LinearSVC

# Создаем и обучаем модель
model = LinearSVC(class_weight='balanced', max_iter=5000, random_state=42)
model.fit(X_train_combined, y_train)

# Сохраняем модель и масштабировщик
joblib.dump(model, 'svm_model.joblib')
joblib.dump(scaler, 'scaler.joblib')


['scaler.joblib']

In [17]:
import joblib

# Загружаем масштабировщик
scaler = joblib.load('scaler.joblib')

# Загружаем модель
model = joblib.load('svm_model.joblib')


In [19]:
import pandas as pd
import re

# Пример новых данных
new_data = pd.DataFrame({
    'text': [
        "The deceased had a large estate worth 500000 dollars.",
        "Small debts and minor funeral expenses were reported."
    ],
    'estate_worth': [500000, 20000],
    'debts': [10000, 500],
    'funeral_expenses': [15000, 1000],
    'bequests': [300000, 5000]
})

# Функция очистки текста (как при обучении)
def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'[^a-z0-9\s]', '', text)
    return text

# Очистка текста
new_data['text_clean'] = new_data['text'].apply(preprocess_text)

# Извлекаем числовые признаки из новых данных
new_data_num_features = new_data[['estate_worth', 'debts', 'funeral_expenses', 'bequests']]

# Векторизуем текст с помощью ранее обученного vectorizer
X_text_new_tfidf = vectorizer.transform(new_data['text_clean'])

# Масштабируем числовые признаки
X_num_new_scaled = scaler.transform(new_data_num_features)

# Объединяем текстовые и числовые признаки
from scipy.sparse import hstack
X_new_combined = hstack([X_text_new_tfidf, X_num_new_scaled])

# Предсказание с помощью загруженной модели
y_new_pred = model.predict(X_new_combined)

# Вывод результатов
for i, pred in enumerate(y_new_pred):
    label = 'Low Bequest' if pred == 0 else 'High Bequest'
    print(f"Sample {i+1} prediction: {label}")


Sample 1 prediction: High Bequest
Sample 2 prediction: Low Bequest


In [20]:
# Коэффициенты модели для каждого признака
print("Коэффициенты модели:", model.coef_)

# Свободный член (сдвиг)
print("Свободный член (intercept):", model.intercept_)


Коэффициенты модели: [[-0.04079304 -0.04079304 -0.04079304  0.02483528 -0.06602843  0.09702109
   4.84206366]]
Свободный член (intercept): [0.14472777]


In [ ]:
from sklearn.calibration import CalibratedClassifierCV
from sklearn.svm import LinearSVC

# Оборачиваем LinearSVC в калибровщик
calibrated_svc = CalibratedClassifierCV(LinearSVC(class_weight='balanced', random_state=42, max_iter=5000), cv=5)

# Обучаем калиброванную модель
calibrated_svc.fit(X_train_combined, y_train)

# Предсказание вероятностей для теста
y_proba = calibrated_svc.predict_proba(X_test_combined)

print("Вероятности принадлежности к классам:\n", y_proba)

# Предсказание классов по максимальной вероятности
y_pred = calibrated_svc.predict(X_test_combined)
